## Integrantes do grupo:

**(Nome do aluno) - (R.A)**

Luiz Fernando Ferreira Berber - 817123421

Lucas Alves de Sousa - 817116864

Matheus Henrique Silva Paiva - 817123241 

Mateus Sampaio de Souza - 817123420

Pedro Jacó Cunha Mesquita - 817124121

In [ ]:
!pip install numpy
!pip install pandas
!pip install regex
!pip install mysql-connector-python

In [ ]:
import numpy as np
import pandas as pd
import mysql.connector
import regex

from mysql.connector import Error
from pathlib import Path

In [ ]:
def get_files_from_path(filepath):
    path = Path(filepath)
    files = []

    for childs in path.iterdir():
        if childs.is_file(): files.append(childs.parts[len(childs.parts) - 1])

    return files

In [ ]:
def create_connection(host_name, user_name, user_password):
    connection = None
    
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [ ]:
def execute_query(connection, query):
    cursor = connection.cursor()
    
    try:
        cursor.execute(query)
        connection.commit()
    except Error as e:
        print(f"The error '{e}' occurred")

In [ ]:
def insert_dimensao_orgao(dados_despesa, connection):
    dimensao_orgao = pd.DataFrame({
        'cod_orgao_superior':dados_despesa['Código Órgão Superior'],
        'nome_orgao_superior':dados_despesa['Nome Órgão Superior'],
        'cod_orgao_subordinado':dados_despesa['Código Órgão Subordinado'],
        'nome_orgao_subordinado':dados_despesa['Nome Órgão Subordinado'],
        'cod_unidade_orcamentaria':dados_despesa['Código Unidade Orçamentária'],
        'nome_unidade_orcamentaria':dados_despesa['Nome Unidade Orçamentária']
    })

    for i in range(dimensao_orgao.shape[0]):
        insert_dimensao_orgao = f"""
        INSERT INTO orcamento_publico.TD_ORGAO
        (
            COD_ORGAO_SUPERIOR
        ,	NOME_ORGAO_SUPERIOR
        ,	COD_ORGAO_SUBORDINADO
        ,	NOME_ORGAO_SUBORDINADO
        ,	COD_UNIDADE_ORCAMENTARIA
        ,	NOME_UNIDADE_ORCAMENTARIA
        ,	DATA_CRIACAO
        )
        VALUES
        (
            {dimensao_orgao.loc[i]['cod_orgao_superior']}
        ,   '{dimensao_orgao.loc[i]['nome_orgao_superior']}'
        ,   {dimensao_orgao.loc[i]['cod_orgao_subordinado']}
        ,   '{dimensao_orgao.loc[i]['nome_orgao_subordinado']}'
        ,   {dimensao_orgao.loc[i]['cod_unidade_orcamentaria']}
        ,   '{dimensao_orgao.loc[i]['nome_unidade_orcamentaria']}'
        ,   NOW()
        );"""

        execute_query(connection, insert_dimensao_orgao)

In [ ]:
def insert_dimensao_programa(dados_despesa, connection):
    dimensao_programa = pd.DataFrame({
        'cod_programa_orcamentario':dados_despesa['Código Programa Orçamentário'],
        'nome_programa_orcamentario':dados_despesa['Nome Programa Orçamentário'],
        'cod_acao':dados_despesa['Código Ação'],
        'nome_acao':dados_despesa['Nome Ação']
    })
    
    for i in range(dimensao_programa.shape[0]):
        insert_dimensao_programa = f"""
        INSERT INTO orcamento_publico.TD_PROGRAMA
        (
            COD_PROGRAMA_ORCAMENTARIO
        ,	NOME_PROGRAMA_ORCAMENTARIO
        ,	COD_ACAO
        ,	NOME_ACAO
        ,	DATA_CRIACAO
        )
        VALUES
        (
         {dimensao_programa.loc[i]['cod_programa_orcamentario']}
        ,'{dimensao_programa.loc[i]['nome_programa_orcamentario']}'
        ,{dimensao_programa.loc[i]['cod_acao']}
        ,'{dimensao_programa.loc[i]['nome_acao']}'
        ,NOW()
        );
        """
        
        execute_query(connection, insert_dimensao_programa)

In [ ]:
def insert_dimensao_data(mes, ano, connection):
    insert_dimensao_data = f"""
    INSERT INTO orcamento_publico.TD_DATA
    (
        MES
    ,	ANO
    ,	DATA_CRIACAO
    )
    VALUES
    (
        {mes}
    ,   {ano}
    ,   NOW()
    );
    """
    
    execute_query(connection, insert_dimensao_data)

In [ ]:
mysql_host_name = '127.0.0.1'
mysql_user_name = 'mateus'
mysql_password = 'root123'

In [ ]:
connection = create_connection(mysql_host_name, mysql_user_name, mysql_password)

In [ ]:
path_dados = '../Datas'
path_despesa = f'{path_dados}/Despesas'
path_orcamento_despesas = f'{path_dados}/OrcamentoDespesas'

In [ ]:
arquivos = get_files_from_path(path_despesa)

In [ ]:
for arquivo in arquivos:
    dados_despesa = pd.read_csv(f'{path_despesa}/{arquivo}', sep=';', encoding='mbcs')
    
    match = regex.match('(?P<ano>\d{4})(?P<mes>\d{2})', arquivo)
    ano = match.group('ano')
    mes = match.group('mes')
    
    insert_dimensao_data(mes, ano, connection)
    insert_dimensao_orgao(dados_despesa, connection)
    insert_dimensao_programa(dados_despesa, connection)
    break